In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import time
import random
import re

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Check if torch will use Apple Silicon GPU

if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

In [ ]:
movielens_data_dir = "ml-32m"

In [ ]:
num_ratings_to_read = 35_000_000

df_ratings = pd.read_csv(movielens_data_dir + '/ratings.csv', nrows=num_ratings_to_read)

In [ ]:
len(df_ratings)

In [ ]:
# clean the ratings data
df_ratings = df_ratings.dropna()
df_ratings['movieId'] = df_ratings['movieId'].astype(int, copy=False)

In [ ]:
df_ratings.head(5)

In [ ]:
df_movies = pd.read_csv(movielens_data_dir + '/movies.csv')

In [ ]:
df_movies.head(3)

In [ ]:
df_tags = pd.read_csv(movielens_data_dir + '/tags.csv')

In [ ]:
df_tags['tag'] = df_tags['tag'].str.lower()
df_tags['tag'] = df_tags['tag'].str.strip()

In [ ]:
df_tags.head(3)

In [ ]:
counts = df_tags.groupby('tag').size().reset_index(name='count')
df_tags_sorted_counts = counts.sort_values(by='count', ascending=False)

MIN_NUM_TAGS = 1000
df_tags_final_counts = df_tags_sorted_counts[df_tags_sorted_counts['count'] > MIN_NUM_TAGS]  
df_tags_final_counts.head(3)

In [ ]:
# Get the final tags we want to use as movie features
final_movie_tags = set(df_tags_final_counts['tag'].tolist())
len(final_movie_tags)

# Movie Feature Processing

In [ ]:
# let's only work with movies with enough ratings.

min_ratings_per_movie = 3_000

# get the number of ratings per movie
df_movies_to_num_ratings = df_ratings.groupby('movieId', as_index=False)['rating'].count()
print("total movies in corpus: ", len(df_movies_to_num_ratings))

df_movies_to_num_ratings = df_movies_to_num_ratings.sort_values(by=['rating'], ascending=False)
df_movies_to_num_ratings = df_movies_to_num_ratings[df_movies_to_num_ratings['rating'] > min_ratings_per_movie]
print("movies with enough ratings: ", len(df_movies_to_num_ratings))

# get list of the top movies by number of ratings.
top_movies = df_movies_to_num_ratings.movieId.tolist()

In [ ]:
# keep a map of movieId to number of ratings.
movieId_to_num_ratings = {}
movieId_list = df_movies_to_num_ratings.movieId.tolist()
rating_list = df_movies_to_num_ratings.rating.tolist()
for i in range(len(movieId_list)):
  movieId_to_num_ratings[movieId_list[i]] = rating_list[i]

In [ ]:
# map movieId to title
movieId_to_title = {}
title_to_movieId = {}

# map moveiId to year
movieId_to_year = {}

movieId_list = df_movies.movieId.tolist()
title_list = df_movies.title.tolist()

for i in range(len(movieId_list)):
  movieId = movieId_list[i]
  title = title_list[i]

  movieId_to_title[movieId] = title
  title_to_movieId[title] = movieId

  match = re.search(r"\(\d+\)\s*$", title)
  if match:
    year = title[match.start()+1:match.end()-1]
    movieId_to_year[movieId] = year
  else:
    movieId_to_year[movieId] = -1

In [ ]:
# keep track of the number of unique year values we see in movieId_to_year
year_to_num_movies = {}
for movieId,year in movieId_to_year.items():
  if year not in year_to_num_movies:
    year_to_num_movies[year] = 0
  year_to_num_movies[year] += 1

In [ ]:
# print the top movies
for movieId in top_movies[0:10]:
  print(movieId, movieId_to_title[movieId], movieId_to_num_ratings[movieId])

In [ ]:
# map movieId to list of genres for that movie
genres = set()
movieId_to_genres = {}

movieId_list = df_movies.movieId.tolist()
genre_list = df_movies.genres.tolist()

for i in range(len(movieId_list)):
  movieId = movieId_list[i]
  if movieId not in top_movies:
    continue

  movieId_to_genres[movieId] = set()

  for genre in genre_list[i].split('|'):
    genres.add(genre)
    movieId_to_genres[movieId].add(genre)

In [ ]:
movieId_to_genres[title_to_movieId['Matrix, The (1999)']]

In [ ]:
# map movieId to the counts of tags for that movie
movieId_to_tag_to_count = {}

movieId_list = df_tags.movieId.tolist()
tag_list = df_tags.tag.tolist()

for i in range(len(movieId_list)):
  movieId = movieId_list[i]
  if movieId not in top_movies:
    continue
      
  if movieId not in movieId_to_tag_to_count:
    movieId_to_tag_to_count[movieId] = {}

  tag = tag_list[i]
  if tag not in final_movie_tags:
    continue
  if tag not in movieId_to_tag_to_count[movieId]:
    movieId_to_tag_to_count[movieId][tag] = 0

  movieId_to_tag_to_count[movieId][tag] += 1

In [ ]:
movieId_to_tag_to_count[title_to_movieId['Matrix, The (1999)']]

In [ ]:
# for every movie, get the avg rating
df_movies_to_avg_rating = df_ratings.groupby('movieId', as_index=False)['rating'].mean()

movieId_to_avg_rating = {}

movieId_list = df_movies_to_avg_rating.movieId.tolist()
rating_list = df_movies_to_avg_rating.rating.tolist()
for i in range(len(movieId_list)):
  if movieId_list[i] not in top_movies: continue
  movieId_to_avg_rating[movieId_list[i]] = rating_list[i]

In [ ]:
# build ITEM movieId embedding mapping
item_emb_movieId_to_i = {s:i for i,s in enumerate(top_movies)}
item_emb_i_to_movieId = {i:s for s,i in item_emb_movieId_to_i.items()}

In [ ]:
# build ITEM genre feature context
genre_to_i = {s:i for i,s in enumerate(genres)}
i_to_genre = {i:s for s,i in genre_to_i.items()}

In [ ]:
# build ITEM year embedding mapping
year_to_i = {s:i for i,s in enumerate(year_to_num_movies.keys())}
i_to_year = {i:s for s,i in year_to_i.items()}

In [ ]:
# build ITEM tag feature context
tag_to_i = {s:i for i,s in enumerate(final_movie_tags)}
i_to_tag = {i:s for s,i in tag_to_i.items()}

# User Feature Processing

In [ ]:
# every user will have a feature context that will mostly be their watch history.
# instead of using every movie in the corpus, we can use a smaller subset.
# this helps with memory issues.
num_movies_for_user_context = 250
user_context_movies = top_movies[:num_movies_for_user_context]

In [ ]:
df_ratings_final = df_ratings[df_ratings.movieId.isin(top_movies)]
df_ratings_final = df_ratings_final.sort_values(['userId', 'timestamp'], ascending=[True, True])

In [ ]:
df_ratings_final.head(3)

In [ ]:
# aggregate dataframe down into one row per user and list of their movies and ratings.
df_ratings_final = df_ratings_final.groupby('userId').agg(
    {'movieId': lambda x: list(x),
     'rating': lambda y: list(y),
     'timestamp': lambda z: list(z)
    }).reset_index()

In [ ]:
df_ratings_final.head(3)

In [ ]:
# build the USER context
user_context_size = len(user_context_movies) + len(genres)

user_context_movieId_to_i = {s:i for i,s in enumerate(list(user_context_movies))}
user_context_i_to_movieId = {i:s for s,i in user_context_movieId_to_i.items()}

user_context_genre_avg_rating_to_i = {s:i+len(user_context_movies) for i,s in enumerate(list(genres))}
user_context_i_to_genre_avg_rating = {i:s for s,i in user_context_genre_avg_rating_to_i.items()}

# Generate Training Examples

In [ ]:
# simulate training examples by masking out some of the user's watched movies from their context, and using them as labels.
# we do not want the 'movie to predict' in their watch history, as we are trying to simulate the following:
# given the user's watch history, what would they rate this new movie?
# NOTE: this is not the same as a train/test split. This is just simulating how training examples would look like on a movie platform.

percent_ratings_as_watch_history = 0.9

min_ratings_per_user = 20 # ignore users with too few movie watches
max_ratings_per_user = 500 # ignore users with way too many movie watches
too_few_ratings = 0
too_many_ratings = 0

user_to_movie_to_rating_WATCH_HISTORY = {}
user_to_movie_to_rating_LABEL = {}
user_to_movie_to_timestamp_LABEL = {}

# loop over each column as this is much, much faster than going row by row.
user_list = df_ratings_final['userId'].tolist()
movieId_list_list = df_ratings_final['movieId'].tolist()
rating_list_list = df_ratings_final['rating'].tolist()
timestamp_list_list = df_ratings_final['timestamp'].tolist()

for i in range(len(user_list)):
  userId = user_list[i]
  movieId_list = movieId_list_list[i]
  rating_list = rating_list_list[i]
  timestamp_list = timestamp_list_list[i]

  num_rated_movies = len(movieId_list)

  # ignore users with too few or too many ratings.
  if num_rated_movies < min_ratings_per_user:
    too_few_ratings += 1
    continue
  if num_rated_movies > max_ratings_per_user:
    too_many_ratings += 1
    continue

  # set up training example maps.
  user_to_movie_to_rating_WATCH_HISTORY[userId] = {}
  user_to_movie_to_rating_LABEL[userId] = {}
  user_to_movie_to_timestamp_LABEL[userId] = {}

  # put earlier watched movies into user's watch history (features) and leave later watched movies as labels to predict.
  rated_movies = list(zip(movieId_list, rating_list, timestamp_list))
  for movieId,rating,timestamp in rated_movies[:int(num_rated_movies * percent_ratings_as_watch_history)]:
    user_to_movie_to_rating_WATCH_HISTORY[userId][movieId] = rating
  for movieId,rating,timestamp in rated_movies[int(num_rated_movies * percent_ratings_as_watch_history):]:
    user_to_movie_to_rating_LABEL[userId][movieId] = rating
    user_to_movie_to_timestamp_LABEL[userId][movieId] = timestamp

In [ ]:
len(user_list), len(user_to_movie_to_rating_WATCH_HISTORY.keys()), too_few_ratings, too_many_ratings

In [ ]:
# for every user, get their avg rating.
# this will help us debias each user's rating.
user_to_avg_rating = {}

# NOTE: only use ratings from their synthetic watch history.
for user in user_to_movie_to_rating_WATCH_HISTORY.keys():
  user_to_avg_rating[user] = 0
  for movieId in user_to_movie_to_rating_WATCH_HISTORY[user].keys():
    user_to_avg_rating[user] += user_to_movie_to_rating_WATCH_HISTORY[user][movieId]

  user_to_avg_rating[user] /= len(user_to_movie_to_rating_WATCH_HISTORY[user].keys())

In [ ]:
user_to_avg_rating[1]

In [ ]:
# for every user, get the avg rating for every genre
user_to_genre_to_stat = {}

# NOTE: only use ratings from their synthetic watch history.
for user in user_to_movie_to_rating_WATCH_HISTORY.keys():
  user_to_genre_to_stat[user] = {}
  for movieId in user_to_movie_to_rating_WATCH_HISTORY[user].keys():
    for genre in movieId_to_genres[movieId]:
      if genre not in user_to_genre_to_stat[user]:
        user_to_genre_to_stat[user][genre] = {
            'NUM_RATINGS': 0,
            'SUM_RATINGS': 0,
        }

      user_to_genre_to_stat[user][genre]['NUM_RATINGS'] += 1
      user_to_genre_to_stat[user][genre]['SUM_RATINGS'] += user_to_movie_to_rating_WATCH_HISTORY[user][movieId]

for user in user_to_genre_to_stat.keys():
  for genre in user_to_genre_to_stat[user].keys():
    num_ratings = user_to_genre_to_stat[user][genre]['NUM_RATINGS']
    sum_ratings = user_to_genre_to_stat[user][genre]['SUM_RATINGS']
    user_to_genre_to_stat[user][genre]['AVG_RATING'] = sum_ratings / num_ratings


In [ ]:
# for every user, create the training example user context vector
# 0:num_user_context_movies -> user's watch history
# num_user_context_movies:num_user_context_movies+num_genres -> user's genre avg rating (debiased using user's avg rating)
user_to_context = {}
for user in user_to_movie_to_rating_WATCH_HISTORY.keys():
  context = [0.0] * user_context_size

  for movieId in user_to_movie_to_rating_WATCH_HISTORY[user].keys():
    if movieId in user_context_movies:
      # note, we debias the rating so if the rating is under the user's avg rating,
      # it will hopefully count as negative strength for predicting similar movies.
      # vice-versa for a rating above the user's average.
      context[user_context_movieId_to_i[movieId]] = float(user_to_movie_to_rating_WATCH_HISTORY[user][movieId] - user_to_avg_rating[user])

  for genre in user_to_genre_to_stat[user].keys():
    # add the user's avg rating for this genre debiased using their actual avg rating from all movies
    context[user_context_genre_avg_rating_to_i[genre]] = float(user_to_genre_to_stat[user][genre]['AVG_RATING'] - user_to_avg_rating[user])

  user_to_context[user] = context

In [ ]:
# for every movie, create a training example feature GENRE context vector lookup
# it will contain the movie's genres.
movieId_to_genre_context = {}
for movieId in top_movies:
  genre_context = [0.0] * len(genres)

  for genre in movieId_to_genres[movieId]:
    genre_context[genre_to_i[genre]] = float(1.0)

  movieId_to_genre_context[movieId] = genre_context

In [ ]:
# for every movie, create a training example feature TAG context vector lookup
# it will contain the movie's tags.
movieId_to_tag_context = {}
for movieId in top_movies:
  tag_context = [0.0] * len(final_movie_tags)

  for tag,count in movieId_to_tag_to_count[movieId].items():
    tag_context[tag_to_i[tag]] = float(count)

  movieId_to_tag_context[movieId] = tag_context

# Build Datasets

In [ ]:
# Prepare the buckets for our timestamps so we can use them as categorical features
timestamp_num_bins = 1500
timestamp_bins = torch.tensor(np.linspace(df_ratings['timestamp'].min(), df_ratings['timestamp'].max(), timestamp_num_bins))

In [ ]:
# Build the final Dataset
def build_dataset(users):
  # the user context (i.e. the watch history and genre affinities)
  X = []

  # the timestamp the user watched this movie (the one we will learn to predict rating for)
  timestamp = []

  # the movieID for the movie we will predict rating for.
  # used to lookup the movie embedding to feed into the NN item tower.
  target_movieId = []

  # the feature GENRE context of the movie we will predict the rating for.
  # will also feed into it's own embedding and will be stacked with the embedding above.
  target_movieId_genre_context = []

  # the feature TAG context of the movie we will predict the rating for.
  # will also feed into it's own embedding and will be stacked with the embedding above.
  target_movieId_tag_context = []

  # the year of the movie we will predict the rating for.
  # used to lookup a year embedding to feed into the NN item tower.
  target_movieId_year = []

  # the predicted rating
  Y = []

  # create training examples, one for each movie the user has that we want as a label.
  for user in users:
    for movieId in user_to_movie_to_rating_LABEL[user].keys():
      X.append(user_to_context[user])

      timestamp.append(user_to_movie_to_timestamp_LABEL[user][movieId])

      target_movieId.append(item_emb_movieId_to_i[movieId])

      target_movieId_genre_context.append(movieId_to_genre_context[movieId])

      target_movieId_tag_context.append(movieId_to_tag_context[movieId])

      target_movieId_year.append(year_to_i[movieId_to_year[movieId]])

      # remember to debias the user rating so we can learn to predict if user
      # like/dislike a movie based on their features and the movie features.
      Y.append(float(user_to_movie_to_rating_LABEL[user][movieId] - user_to_avg_rating[user]))

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  target_movieId = torch.tensor(target_movieId)
  target_movieId_genre_context = torch.tensor(target_movieId_genre_context)
  target_movieId_tag_context = torch.tensor(target_movieId_tag_context)
  target_movieId_year = torch.tensor(target_movieId_year)

  # Bucketize the timestamps
  timestamp = torch.bucketize(torch.tensor(timestamp), timestamp_bins, right=False)

  return X,timestamp,Y,target_movieId,target_movieId_genre_context,target_movieId_tag_context,target_movieId_year

In [ ]:
# user users with enough ratings to predict to be useful for model learning.
final_users = []

for user in user_to_movie_to_rating_LABEL.keys():
  num_ratings = len(user_to_movie_to_rating_LABEL[user])

  # having at least 2 watched movies in the user's LABEL split means they have watched at least 20 movies
  # 18 of these movies are used for the user's watch history
  if num_ratings >= 2 and num_ratings < 500:
    final_users.append(user)

len(final_users)

In [ ]:
# split users into train and validation users
percent_users_train = 0.9

random.shuffle(final_users)

train_users = final_users[:int(len(final_users) * percent_users_train)]
validation_users = final_users[int(len(final_users) * percent_users_train):]

In [ ]:
X_train, timestamp_train, Y_train, target_movieId_train, target_movieId_genre_context_train, target_movieId_tag_context_train, target_movieId_year_train = build_dataset(train_users)
X_val, timestamp_val, Y_val, target_movieId_val, target_movieId_genre_context_val, target_movieId_tag_context_val, target_movieId_year_val = build_dataset(validation_users)

In [ ]:
print("train: ", 
      X_train.shape,
      timestamp_train.shape,
      Y_train.shape,
      target_movieId_train.shape,
      target_movieId_genre_context_train.shape,
      target_movieId_tag_context_train.shape,
      target_movieId_year_train.shape)
print("val: ",
      X_val.shape,
      timestamp_val.shape,
      Y_val.shape,
      target_movieId_val.shape,
      target_movieId_genre_context_val.shape,
      target_movieId_tag_context_val.shape,
      target_movieId_year_val.shape)

# Build our Two Tower Model

In [ ]:
'''
user_features   -> u_W1
                        \         
                         --> stack
                        /          \
timestamp_emb   -> t_W1             \
                                     \
                                      \
                                       --> dot_product(user, item) --> prediction
                                      /
                                     /
movie_genres    -> g_W1             /
                                   /
movie_tags      -> t_W1           /
                        ---> stack
movie_year_emb  -> y_W1
                    
movie_id_emb    -> e_W1
'''

class MovieRecommender(nn.Module):
    def __init__(self,
                 genres_len,
                 tags_len,
                 top_movies_len,
                 all_years_len,
                 user_context_size,
                 timestamp_num_bins,
                 item_genre_embedding_size=10,
                 item_tag_embedding_size=10,
                 item_movieId_embedding_size=40,
                 item_year_embedding_size=10,
                 user_feature_embedding_size=60,
                 timestamp_feature_embedding_size=10,
                ):
        """
        Initializes the MovieRecommender model.

        Args:
            genres_len (int): Number of unique genres (dimension of movie genre features).
            tags_len (int): Number of unique tags (dimension of movie tag features).
            top_movies_len (int): Total number of unique movie IDs in the lookup table.
            all_years_len (int): Number of unique movie release years.
            user_context_size (int): Dimension of user context features.
            item_genre_embedding_size (int): Desired embedding size for item genre features.
            item_tag_embedding_size (int): Desired embedding size for item tag features.
            item_movieId_embedding_size (int): Desired embedding size for movie IDs.
            item_year_embedding_size (int): Desired embedding size for the movie year.
            user_feature_embedding_size (int): Desired embedding size for user features.
            timestamp_feature_embedding_size (int): Desired embedding size for the timestamp features.
        """
        super().__init__()
        self.item_genre_tower = nn.Sequential(
            nn.Linear(genres_len, item_genre_embedding_size),
            nn.Tanh()
        )

        self.item_tag_tower = nn.Sequential(
            nn.Linear(tags_len, item_tag_embedding_size),
            nn.Tanh()
        )

        self.item_embedding_lookup = nn.Embedding(top_movies_len, item_movieId_embedding_size)
        self.item_embedding_tower = nn.Sequential(
            nn.Linear(item_movieId_embedding_size, item_movieId_embedding_size),
            nn.Tanh()
        )

        self.year_embedding_lookup = nn.Embedding(all_years_len, item_year_embedding_size)
        self.year_embedding_tower = nn.Sequential(
            nn.Linear(item_year_embedding_size, item_year_embedding_size),
            nn.Tanh()
        )

        self.user_feature_tower = nn.Sequential(
            nn.Linear(user_context_size, user_feature_embedding_size),
            nn.Tanh()
        )

        self.timestamp_embedding_lookup = nn.Embedding(timestamp_num_bins, timestamp_feature_embedding_size)
        self.timestamp_embedding_tower = nn.Sequential(
            nn.Linear(timestamp_feature_embedding_size, timestamp_feature_embedding_size),
            nn.Tanh()
        )

        # Dimension check:
        # user+timestamp == item_embedding+year_embedding+item_genre+item_tag
        expected_combined_item_embedding_size = item_genre_embedding_size + item_tag_embedding_size + item_movieId_embedding_size + item_year_embedding_size
        expected_combined_user_embedding_size = user_feature_embedding_size + timestamp_feature_embedding_size
        if expected_combined_user_embedding_size != expected_combined_item_embedding_size:
            raise ValueError(
                f"User+timestamp embedding size ({expected_combined_user_embedding_size}) must match "
                f"combined item embedding size ({expected_combined_item_embedding_size}) "
                f"for the dot product operation. Please adjust `user_feature_embedding_size`."
            )

        # Apply custom weight initialization to all applicable layers
        self.apply(self._init_weights)

    def _init_weights(self, module):
        """
        Applies a small scale to initial weights for Linear and Embedding layers.
        """
        if isinstance(module, nn.Linear):
            # Using Xavier uniform initialization which is good for Tanh activation
            # The gain parameter scales the initialization.
            torch.nn.init.xavier_uniform_(module.weight, gain=0.01)
            if module.bias is not None:
                # Initialize biases to zero
                torch.nn.init.constant_(module.bias, 0)
        elif isinstance(module, nn.Embedding):
            # For embedding layers, also use Xavier uniform initialization
            torch.nn.init.xavier_uniform_(module.weight, gain=0.01)

    def forward(self, user_contexts, timestamps, movie_genres, movie_tags, years, target_movieId):
        """
        Args:
            user_contexts (torch.Tensor):  Batch of user context features.
                                           Shape: (batch_size, user_context_size)
            timestamps (torch.Tensor):     Batch of timestamp features.
                                           Shape: (batch_size, timestamp_feature_embedding_size)
            movie_genres (torch.Tensor):   Batch of target movie genre features.
                                           Shape: (batch_size, genres_len)
            movie_tags (torch.Tensor):     Batch of target movie tag features.
                                           Shape: (batch_size, tags_len)
            years (torch.Tensor):          Batch of target movie years.
                                           Shape: (batch_size, all_years_len)
            target_movieId (torch.Tensor): Batch of target movie ID indices.
                                           Shape: (batch_size,)
        """
        # Forward pass through the USER tower
        user_embedding = self.user_feature_tower(user_contexts) # Shape: (batch_size, user_feature_embedding_size)

        # Lookup the TIMESTAMP embedding and pass through its non-linear layer
        timestamp_embedding_hidden = self.timestamp_embedding_tower(self.timestamp_embedding_lookup(timestamps)) # Shape: (batch_size, timestamp_feature_embedding_size)

        # Concatenate the USER and TIMESTAMP embeddings together
        # `torch.cat` combines tensors along a given dimension.
        # `dim=1` means concatenate along the feature dimension for each sample in the batch.
        user_embedding_combined = torch.cat((user_embedding, timestamp_embedding_hidden), dim=1)
        # Resulting shape: (batch_size, user_feature_embedding_size + timestamp_feature_embedding_size)

        # Forward pass through the ITEM movie genre tower
        item_genre_embedding = self.item_genre_tower(movie_genres) # Shape: (batch_size, item_genre_embedding_size)

        # Forward pass through the ITEM movie tag tower
        item_tag_embedding = self.item_tag_tower(movie_tags) # Shape: (batch_size, item_tag_embedding_size)

        # Lookup the ITEM movieId embedding and pass through its non-linear layer
        item_embedding_hidden = self.item_embedding_tower(self.item_embedding_lookup(target_movieId)) # Shape: (batch_size, item_movieId_embedding_size)

        # Lookup the ITEM year embedding and pass through its non-linear layer
        year_embedding_hidden = self.year_embedding_tower(self.year_embedding_lookup(years)) # Shape: (batch_size, item_year_embedding_size)

        # Concatenate the two ITEM embeddings together
        item_embedding_combined = torch.cat((item_genre_embedding, item_tag_embedding, item_embedding_hidden, year_embedding_hidden), dim=1)
        # Resulting shape: (batch_size, item_genre_embedding_size + item_tag_embedding_size + item_movieId_embedding_size + item_year_embedding_size)

        # The final prediction is the dot product of the user+timestamp embedding and the combined item embedding.
        # `torch.einsum('ij, ij -> i', A, B)` performs element-wise multiplication
        # and then sums along the 'j' dimension for each 'i'.
        # This is equivalent to `(user_embedding * item_embedding_combined).sum(dim=1)`.
        preds = torch.einsum('ij, ij -> i', user_embedding_combined, item_embedding_combined)

        return preds


# Training Loop

In [ ]:
# --- Training Loop ---

# Define model hyper-parameters
item_genre_embedding_size = 10
item_tag_embedding_size = 40
item_movieId_embedding_size = 40
item_year_embedding_size = 10

# User embedding size + timestamp embedding size must match the sum of item_feature_embedding_size + item_movieId_embedding_size + item_year_embedding_size
user_feature_embedding_size = 90
timestamp_feature_embedding_size = 10

# Instantiate the MovieRecommender model
model = MovieRecommender(
    genres_len=len(genres),
    tags_len=len(final_movie_tags),
    top_movies_len=len(top_movies),
    all_years_len=len(year_to_num_movies),
    user_context_size=user_context_size,
    timestamp_num_bins=timestamp_num_bins,
    item_genre_embedding_size=item_genre_embedding_size,
    item_tag_embedding_size=item_tag_embedding_size,
    item_movieId_embedding_size=item_movieId_embedding_size,
    user_feature_embedding_size=user_feature_embedding_size,
    timestamp_feature_embedding_size=timestamp_feature_embedding_size
)

# Print the total number of trainable parameters in the model
print(f"Number of trainable parameters: {sum(p.nelement() for p in model.parameters() if p.requires_grad)}")

In [ ]:
USE_GPU = False

if USE_GPU:
    mps_device = torch.device("mps")
    
    # Move the model to the GPU
    model.to(mps_device)
    
    # move validation data to MPS
    X_val = X_val.to(mps_device)
    timestamp_val = timestamp_val.to(mps_device)
    target_movieId_genre_context_val = target_movieId_genre_context_val.to(mps_device)
    target_movieId_tag_context_val = target_movieId_tag_context_val.to(mps_device)
    target_movieId_year_val = target_movieId_year_val.to(mps_device)
    target_movieId_val = target_movieId_val.to(mps_device)
    Y_val = Y_val.to(mps_device)
    
    # move training data to MPS
    X_train = X_train.to(mps_device)
    timestamp_train = timestamp_train.to(mps_device)
    target_movieId_genre_context_train = target_movieId_genre_context_train.to(mps_device)
    target_movieId_tag_context_train = target_movieId_tag_context_train.to(mps_device)
    target_movieId_year_train = target_movieId_year_train.to(mps_device)
    target_movieId_train = target_movieId_train.to(mps_device)
    Y_train = Y_train.to(mps_device)
else:
    # move validation data to CPU
    X_val = X_val.cpu()
    timestamp_val = timestamp_val.cpu()
    target_movieId_genre_context_val = target_movieId_genre_context_val.cpu()
    target_movieId_tag_context_val = target_movieId_tag_context_val.cpu()
    target_movieId_year_val = target_movieId_year_val.cpu()
    target_movieId_val = target_movieId_val.cpu()
    Y_val = Y_val.cpu()
    
    # move training data to CPU
    X_train = X_train.cpu()
    timestamp_train = timestamp_train.cpu()
    target_movieId_genre_context_train = target_movieId_genre_context_train.cpu()
    target_movieId_tag_context_train = target_movieId_tag_context_train.cpu()
    target_movieId_year_train = target_movieId_year_train.cpu()
    target_movieId_train = target_movieId_train.cpu()
    Y_train = Y_train.cpu()

In [ ]:
# Set the loss function (Mean Squared Error Loss for regression)
loss_fn = torch.nn.MSELoss()

# Set the optimizer (Stochastic Gradient Descent)
# It will manage the updates to all parameters in `model.parameters()`
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

# Training configuration
minibatch_size = 64
loss_train = [] # To store training loss for each step
loss_val = []   # To store validation loss for full validation runs

log_every = 10_000      # How often to perform a full validation run and log
training_steps = 150_000 # Total training iterations

In [ ]:
print("\nStarting training loop...")

start = time.time()
for i in range(training_steps):
    is_full_val_run = False
    if i % log_every == 0:
        is_full_val_run = True

    # Select data for the current step (training minibatch or full validation set)
    if is_full_val_run:
        # Use full validation set for evaluation
        user_contexts_batch = X_val
        timestamps_batch = timestamp_val
        movie_genres_batch = target_movieId_genre_context_val
        movie_tags_batch = target_movieId_tag_context_val
        movie_years_batch = target_movieId_year_val
        target_movieId_batch = target_movieId_val
        Y_batch = Y_val

        # Set model to evaluation mode (e.g., disables dropout if present)
        model.eval()
        
        with torch.no_grad(): # Disable gradient calculations for validation
            preds = model(user_contexts_batch, timestamps_batch, movie_genres_batch, movie_tags_batch, movie_years_batch, target_movieId_batch)
            output = loss_fn(preds, Y_batch)

        # Store validation loss for this batch
        loss_val.append(output.item()) # Store validation loss
    else:
        # Construct a random minibatch from training data
        ix = torch.randint(0, X_train.shape[0], (minibatch_size,))
        user_contexts_batch = X_train[ix]
        timestamps_batch = timestamp_train[ix]
        movie_genres_batch = target_movieId_genre_context_train[ix]
        movie_tags_batch = target_movieId_tag_context_train[ix]
        movie_years_batch = target_movieId_year_train[ix]
        target_movieId_batch = target_movieId_train[ix]
        Y_batch = Y_train[ix]      

        # Set model to training mode (e.g., enables dropout)
        model.train()

        # Forward pass
        preds = model(user_contexts_batch, timestamps_batch, movie_genres_batch, movie_tags_batch, movie_years_batch, target_movieId_batch)
        output = loss_fn(preds, Y_batch)

        # Backpropagation and update weights
        optimizer.zero_grad() # Clear gradients from the previous step
        output.backward()     # Compute gradients for all parameters
        optimizer.step()      # Update model parameters using the computed gradients

        # Store training loss for this batch
        loss_train.append(output.item())

    # Logging
    if is_full_val_run:
        # print how long in between each full val run
        end = time.time()
        length = end - start
        print("Total time: ", length, "seconds")
        start = time.time()
        
        # Calculate average training loss over the last `log_every` batches
        if i >= log_every:
            avg_train_loss_last_batches = np.mean(loss_train[i-log_every:i])
        else:
            # If not enough training batches collected yet, use the current validation loss
            # (or you could skip logging train loss for the very first validation run)
            avg_train_loss_last_batches = output.item()
        print(f"[TRAIN] i: {i:06d} | avg_loss (last {log_every} batches): {avg_train_loss_last_batches:.4f}")
        print(f"[VAL]   i: {i:06d} | loss: {output.item():.4f}\n")

print("Training complete.")

In [ ]:
loss_train_bucket_means = []
for i in range(0, len(loss_train), log_every):
  loss_train_bucket_means.append(np.mean(loss_train[i:i+log_every]))

plt.plot([i*1000 for i in range(len(loss_train_bucket_means))], loss_train_bucket_means)
plt.plot([i*1000 for i in range(1, len(loss_val))], loss_val[1:])

plt.show()

# Save the Model

In [ ]:
PATH = 'saved_models/20250602.pth'

torch.save(model.state_dict(), PATH)

# Actually Using the Model

In [ ]:
model = MovieRecommender(
    genres_len=len(genres),
    tags_len=len(final_movie_tags),
    top_movies_len=len(top_movies),
    all_years_len=len(year_to_num_movies),
    user_context_size=user_context_size,
    timestamp_num_bins=timestamp_num_bins,
    item_genre_embedding_size=item_genre_embedding_size,
    item_tag_embedding_size=item_tag_embedding_size,
    item_movieId_embedding_size=item_movieId_embedding_size,
    user_feature_embedding_size=user_feature_embedding_size,
    timestamp_feature_embedding_size=timestamp_feature_embedding_size
)
model.load_state_dict(torch.load(PATH, weights_only=True))
model.eval()

In [ ]:
# for every movie, save all its embeddings
movieId_to_embedding = {}

ITEM_EMBEDDING_LOOKUP = model.item_embedding_lookup.weight
YEAR_EMBEDDING_LOOKUP = model.year_embedding_lookup.weight

for movieId in top_movies:
  movieId_to_embedding[movieId] = {}

  item_embedding = ITEM_EMBEDDING_LOOKUP[torch.tensor([item_emb_movieId_to_i[movieId]])]
  movieId_to_embedding[movieId]['MOVIEID_EMBEDDING'] = model.item_embedding_tower(item_embedding)

  year_embedding = YEAR_EMBEDDING_LOOKUP[torch.tensor([year_to_i[movieId_to_year[movieId]]])]
  movieId_to_embedding[movieId]['MOVIE_YEAR_EMBEDDING'] = model.year_embedding_tower(year_embedding)

  movieId_to_embedding[movieId]['MOVIE_GENRE_EMBEDDING'] = model.item_genre_tower(torch.tensor([movieId_to_genre_context[movieId]]))
  movieId_to_embedding[movieId]['MOVIE_TAG_EMBEDDING'] = model.item_tag_tower(torch.tensor([movieId_to_tag_context[movieId]]))

  # compute the combined (concat) item/movie embedding
  item_id_emb = movieId_to_embedding[movieId]['MOVIEID_EMBEDDING']
  item_genre_emb = movieId_to_embedding[movieId]['MOVIE_GENRE_EMBEDDING']
  item_tag_emb = movieId_to_embedding[movieId]['MOVIE_TAG_EMBEDDING']
  iteam_year_emb = movieId_to_embedding[movieId]['MOVIE_YEAR_EMBEDDING']
  movieId_to_embedding[movieId]['MOVIE_EMBEDDING_COMBINED'] = torch.cat((item_genre_emb, item_tag_emb, item_id_emb, iteam_year_emb), dim=1)

In [ ]:
for emb_type in movieId_to_embedding[5952].keys():
  print(movieId_to_embedding[5952][emb_type].shape)

### Viewing Movies in 2D

In [ ]:
plt.figure(figsize=(15,15))
for movieId in top_movies[0:25]:
  i = item_emb_movieId_to_i[movieId]
  plt.scatter(ITEM_EMBEDDING_LOOKUP[i,0].data, ITEM_EMBEDDING_LOOKUP[i,1].data, s=200)
  plt.text(ITEM_EMBEDDING_LOOKUP[i,0].item(), ITEM_EMBEDDING_LOOKUP[i,1].item(), movieId_to_title[movieId][0:20], ha="center", va="center", color='black')
plt.grid('minor')

plt.show()

### Finding Most Similar Movies

In [ ]:
# for every movie, and for every embedding type, find the similary to all other embeddings
# NOTE: can be slow
movieId_to_emb_type_to_similarities = {}

for movieId in top_movies:
  movieId_to_emb_type_to_similarities[movieId] = {}

  # for emb_type in movieId_to_embedding[movieId].keys():
  for emb_type in ['MOVIE_EMBEDDING_COMBINED']:
    emb_to_target_to_dist = {}
    for target_id in top_movies:
      src = movieId_to_embedding[movieId][emb_type].view(-1)
      target = movieId_to_embedding[target_id][emb_type].view(-1)

      distance = torch.sqrt(torch.sum(torch.pow(torch.subtract(src, target), 2), dim=0))
      emb_to_target_to_dist[target_id] = distance.item()
    movieId_to_emb_type_to_similarities[movieId][emb_type] = list(sorted(emb_to_target_to_dist.items(), key=lambda item: item[1]))[0:100]

In [ ]:
titles = [
    'Lord of the Rings: The Return of the King, The (2003)',
    'Star Wars: Episode IV - A New Hope (1977)',
    'Toy Story (1995)',
    'Saving Private Ryan (1998)',
    'Kill Bill: Vol. 1 (2003)',
    'American Pie (1999)',
    'Blair Witch Project, The (1999)',
    'Princess Mononoke (Mononoke-hime) (1997)'
]

emb_type = 'MOVIE_EMBEDDING_COMBINED'

table = '| Movie |'
for i in range(5):
  table += ' Similar {} |'.format(i+1)
table += '\n'
for i in range(5):
  table += '|-----'
table += '|\n'

# Print the top most similar movies
for title in titles:
  movieId = title_to_movieId[title]

  table += '| '
  for target_id, dist in movieId_to_emb_type_to_similarities[movieId][emb_type][0:5+1]:
    table += movieId_to_title[target_id] + ' | '
  table += '\n'

print(table)

### Get Recommendations for Users

In [ ]:
for movieId in user_context_movies:
    print(movieId_to_title[movieId], movieId_to_genres[movieId])

In [ ]:
user_type_to_favorite_genres = {
    'Fantasy Lover': ['Fantasy'],
    'Children\'s Movie Lover': ['Children'],
    'Horror Lover': ['Horror'],
    'Sci-Fi Lover': ['Sci-Fi'],
    'Comedy Lover': ['Comedy'],
    'Romance Lover': ['Romance'],
    'War Movie Lover': ['War'],
    'Old Movie Lover': [],

    # profile for myself
    'Myself': ['Fantasy', 'War', 'Horror', 'Drama', 'Action']
}

user_type_to_worst_genres = {
    'Fantasy Lover': ['Horror', 'Children'],
    'Children\'s Movie Lover': ['Horror', 'Romance', 'Drama'],
    'Horror Lover': ['Children'],
    'Sci-Fi Lover': ['Romance', 'Children'],
    'Comedy Lover': ['Children'],
    'Romance Lover': ['Children', 'Horror'],
    'War Movie Lover': ['Children'],
    'Old Movie Lover': [],
    
    # profile for myself
    'Myself': ['Romance']
}

user_type_to_favorite_movies = {
    'Fantasy Lover': [
        'Lord of the Rings: The Fellowship of the Ring, The (2001)',
        'Lord of the Rings: The Two Towers, The (2002)',
        'Gladiator (2000)',
        '300 (2007)',
        'Braveheart (1995)'
        ],
    'Children\'s Movie Lover': [
        'Toy Story 2 (1999)',
        'Finding Nemo (2003)',
        'Monsters, Inc. (2001)'
        ],
    'Horror Lover': [
        'Blair Witch Project, The (1999)',
        'Silence of the Lambs, The (1991)',
        'Sixth Sense, The (1999)'
        ],
    'Sci-Fi Lover': [
        'Star Wars: Episode V - The Empire Strikes Back (1980)',
        'Matrix, The (1999)',
        'Terminator, The (1984)'
        ],
    'Comedy Lover': [
        'American Pie (1999)',
        'Dumb & Dumber (Dumb and Dumber) (1994)',
        'Austin Powers: The Spy Who Shagged Me (1999)',
        'Big Lebowski, The (1998)'
      ],
    'Romance Lover': [
        'Shakespeare in Love (1998)',
        'There\'s Something About Mary (1998)',
        'Sense and Sensibility (1995)'
    ],
    'War Movie Lover': [
        'Saving Private Ryan (1998)',
        'Apocalypse Now (1979)',
        'Full Metal Jacket (1987)'
    ],
    'Old Movie Lover': [
        '12 Angry Men (1957)',
        'Graduate, The (1967)',
        'Wizard of Oz, The (1939)',
        'Psycho (1960)',
        'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)',
    ],
    
    # profile for myself
    'Myself': [
        'Lord of the Rings: The Fellowship of the Ring, The (2001)',
        'Lord of the Rings: The Two Towers, The (2002)',
        'Lord of the Rings: The Return of the King, The (2003)',
        '300 (2007)',
        'Saving Private Ryan (1998)',
        'Kill Bill: Vol. 1 (2003)',
    ]
}

value_for_favorite_genre_avg_rating = float(5.0) # make much larger than usual to make liked genres have more emphasis
value_for_disliked_genre_avg_rating = float(-2.0)
value_for_favorite_movie_rating = float(2.0)

user_to_inference_context = {}

for user_type in user_type_to_favorite_genres.keys():
  inference_user_context = [0.0] * user_context_size

  # set genres the user likes (avg rating + ratio)
  for genre in user_type_to_favorite_genres[user_type]:
    inference_user_context[user_context_genre_avg_rating_to_i[genre]] = value_for_favorite_genre_avg_rating

  # set genres that the user dislikes (avg rating)
  for genre in user_type_to_worst_genres[user_type]:
    inference_user_context[user_context_genre_avg_rating_to_i[genre]] = value_for_disliked_genre_avg_rating

  # set the user's favorite movies.
  for title in user_type_to_favorite_movies[user_type]:
    movieId = title_to_movieId[title]
    inference_user_context[user_context_movieId_to_i[movieId]] = value_for_favorite_movie_rating

  user_to_inference_context[user_type] = inference_user_context

In [ ]:
user_to_top_recs = {}

for user_type in user_to_inference_context.keys():

  X_inference = torch.tensor([user_to_inference_context[user_type]])
  user_embedding_inference = model.user_feature_tower(X_inference)

  # we also need a timestamp in order to run inference, so let's use the max timestamp from all our training data
  timestamp_inference = torch.bucketize(torch.tensor([df_ratings['timestamp'].max()]), timestamp_bins, right=False)
  timestamp_embedding = model.timestamp_embedding_tower(model.timestamp_embedding_lookup(timestamp_inference))

  # combine the user and timestamp embeddings just as we do when training
  user_and_timestamp_combined_embedding = torch.cat((user_embedding_inference, timestamp_embedding), dim=1)

  movieId_to_pred_score = {}
  for movieId in top_movies:
    # we already have the combined item embedding for every movie to make inference easier.
    item_embedding_combined_inference = movieId_to_embedding[movieId]['MOVIE_EMBEDDING_COMBINED']
    preds = torch.einsum('ij, ij -> i', user_and_timestamp_combined_embedding, item_embedding_combined_inference)
    movieId_to_pred_score[movieId] = preds

  top_recs = []
  show_top_recs = True
  for movieId, pred_score in list(sorted(movieId_to_pred_score.items(), key=lambda item: item[1], reverse=show_top_recs)):
    if len(top_recs) >= 10: break
    if movieId_to_title[movieId] not in user_type_to_favorite_movies[user_type]:
      top_recs.append(movieId)
  user_to_top_recs[user_type] = top_recs

In [ ]:
for user_type in user_to_top_recs.keys():
  print("Hello, " + user_type)
  print("Because you like: [" + ','.join(user_type_to_favorite_genres[user_type]) + ']')
  print("And hate: [" + ','.join(user_type_to_worst_genres[user_type]) + ']')
  print("And enjoyed these movies:")
  for title in user_type_to_favorite_movies[user_type]:
    print(title)
  print()

  print("You should watch:")
  for movieId in user_to_top_recs[user_type]:
    print(movieId_to_title[movieId])
  print()

In [ ]:
# sanity check - make sure we aren't just recommending the higest rated movies
# NOTE: this is an extremely common problem in rec systems as the model learns
# to play it safe and just recommend what almost is highly rated.
for movieId, avg_rating in sorted(movieId_to_avg_rating.items(), key=lambda item: item[1], reverse=True)[0:10]:
  print(movieId_to_title[movieId], avg_rating)

In [ ]:
# Finding the 'most representative' movies for a specfic GENRE
genre = 'Romance'
genre_context = [0.0] * len(genres)
genre_context[genre_to_i[genre]] = float(1.0)
genre_context_emb = model.item_genre_tower(torch.tensor([genre_context]))

movieId_to_dist = {}
for movieId in top_movies:
  distance = torch.sqrt(torch.sum(torch.pow(torch.subtract(movieId_to_embedding[movieId]['MOVIE_GENRE_EMBEDDING'].view(-1), genre_context_emb.view(-1)), 2), dim=0))
  movieId_to_dist[movieId] = distance.item()

for movieId,dist in list(sorted(movieId_to_dist.items(), key=lambda item: item[1]))[0:10]:
    print(movieId_to_title[movieId])

In [ ]:
movieId_to_tag_to_count[title_to_movieId['Saw (2004)']]

In [ ]:
# Finding the 'most representative' movies for a specfic TAG
tags = ['horror', 'gore', 'torture']
tags = ['dystopia', 'artificial intelligence', 'martial arts']
tag_context = [0.0] * len(final_movie_tags)
for tag in tags:
  tag_context[tag_to_i[tag]] = float(10.0)
tag_context_emb = model.item_tag_tower(torch.tensor([tag_context]))

movieId_to_dist = {}
for movieId in top_movies:
  distance = torch.sqrt(torch.sum(torch.pow(torch.subtract(movieId_to_embedding[movieId]['MOVIE_TAG_EMBEDDING'].view(-1), tag_context_emb.view(-1)), 2), dim=0))
  movieId_to_dist[movieId] = distance.item()

for movieId,dist in list(sorted(movieId_to_dist.items(), key=lambda item: item[1]))[0:10]:
    print(movieId_to_title[movieId])